In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
#export

import operator
from functools import partial
from collections import deque
from numpy import array, ndarray
import pandas as pd
from typing import Any, Collection, Callable, NewType, List, Union, TypeVar, Optional, Generator, Iterable
import itertools
from pandas.api.types import is_categorical_dtype,is_numeric_dtype
from numpy import array,ndarray
from scipy import ndimage
from IPython.core.debugger import set_trace
import numpy as np
try:
    from types import WrapperDescriptorType,MethodWrapperType,MethodDescriptorType
except ImportError:
    WrapperDescriptorType = type(object.__init__)
    MethodWrapperType = type(object().__str__)
    MethodDescriptorType = type(str.join)
from types import BuiltinFunctionType,BuiltinMethodType,MethodType,FunctionType

pd.options.display.max_colwidth = 600
NoneType = type(None)
string_classes = (str,bytes)

In [3]:
#export

def all_equal(a,b):
    "Compares whether `a` and `b` are the same length and have the same contents"
    if not is_iter(b): return False
    return all(equals(a_,b_) for a_,b_ in itertools.zip_longest(a,b))

In [4]:
#export

def equals(a,b):
    "Compares `a` and `b` for equality; supports sublists, tensors and arrays too"
    if one_is_instance(a,b,type): return a==b
    if hasattr(a, '__array_eq__'): return a.__array_eq__(b)
    if hasattr(b, '__array_eq__'): return b.__array_eq__(a)
    cmp = (np.array_equal if one_is_instance(a, b, ndarray       ) else
           operator.eq    if one_is_instance(a, b, (str,dict,set)) else
           all_equal      if is_iter(a) or is_iter(b) else
           operator.eq)
    return cmp(a,b)

In [5]:
#export

def test(a, b, cmp, cname=None):
    'assert cmp(a,b)'
    if cname is None: cname=cmp.__name__
    assert cmp(a, b), f'{cname}: \na\nb'

In [ ]:
#export
def is_iter(o):
    "Test whether `o` can be used in a `for` loop"
    #Rank 0 tensors in PyTorch are not really iterable
    return isinstance(o, (Iterable,Generator)) and getattr(o,'ndim',1)

def is_coll(o):
    "Test whether `o` is a collection (i.e. has a usable `len`)"
    #Rank 0 tensors in PyTorch do not have working `len`
    return hasattr(o, '__len__') and getattr(o,'ndim',1)

def all_equal(a,b):
    "Compares whether `a` and `b` are the same length and have the same contents"
    if not is_iter(b): return False
    return all(equals(a_,b_) for a_,b_ in itertools.zip_longest(a,b))

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

def noops(self, x=None, *args, **kwargs):
    "Do nothing (method)"
    return x

def one_is_instance(a, b, t): return isinstance(a,t) or isinstance(b,t)

def equals(a,b):
    "Compares `a` and `b` for equality; supports sublists, tensors and arrays too"
    if one_is_instance(a,b,type): return a==b
    if hasattr(a, '__array_eq__'): return a.__array_eq__(b)
    if hasattr(b, '__array_eq__'): return b.__array_eq__(a)
    cmp = (np.array_equal if one_is_instance(a, b, ndarray       ) else
           operator.eq    if one_is_instance(a, b, (str,dict,set)) else
           all_equal      if is_iter(a) or is_iter(b) else
           operator.eq)
    return cmp(a,b)

In [6]:
#export

test_eq= partial(test,cmp=operator.eq, cname='==')
test_ne= partial(test,cmp=operator.ne)

In [7]:
# tests

test_eq(5, 5.)
test_eq('narain', 'nar'+'ain')
test_eq('aaaa', 'a'*4)
test_ne('narain', 'mon')
test_ne(deque([5,4,3,2]), 'na')

In [8]:
#export

def test_type_eq(a, b):
    test_eq(a,b)
    test_eq(type(a), type(b))
    if isinstance(a, (list, tuple)): test_eq(map(type, a), map(type, b))

In [9]:
#export

def is_iter(a):
    return isinstance(a, (Iterable, Generator)) and getattr(a, 'ndim', 1)

In [10]:
#export

def is_coll(a):
    return hasattr(a, '__len__') and getattr(a, 'ndim', 1)

In [2]:
#export
def test_shuffled(a,b):
    "`test` that `a` and `b` are shuffled versions of the same sequence of items"
    test_ne(a, b)
    test_eq(Counter(a), Counter(b))

In [3]:
#export
def test_close(a,b,eps=1e-5):
    "`test` that `a` is within `eps` of `b`"
    test(a,b,partial(is_close,eps=eps),'close')

In [4]:
#export
def test_is(a,b):
    "`test` that `a is b`"
    test(a,b,operator.is_, 'is')

In [5]:
#export
def test_stdout(f, exp, regex=False):
    "Test that `f` prints `exp` to stdout, optionally checking as `regex`"
    s = io.StringIO()
    with redirect_stdout(s): f()
    if regex: assert re.search(exp, s.getvalue()) is not None
    else: test_eq(s.getvalue(), f'{exp}\n' if len(exp) > 0 else '')

In [6]:
#export
def test_warns(f, show=False):
    with warnings.catch_warnings(record=True) as w:
        f()
        test_ne(len(w), 0)
        if show: 
            for e in w: print(f"{e.category}: {e.message}")

In [7]:
#export
def test_fig_exists(ax):
    "Test there is a figure displayed in `ax`"
    assert ax and len(np.frombuffer(ax.figure.canvas.tostring_argb(), dtype=np.uint8))

In [1]:
!python notebook2script.py 00_test.ipynb

Traceback (most recent call last):
  File "notebook2script.py", line 3, in <module>
    import json,fire,re
ModuleNotFoundError: No module named 'fire'
